In [ ]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [ ]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [ ]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = '.\\Outputs'

In [ ]:
hui_df = pd.DataFrame.spatial.from_featureclass(r'.\Outputs\hui_for_web.gdb\hui_pts_web_version')

In [ ]:
hui_df.loc[hui_df['CITY'] == '', 'CITY'] = np.nan
hui_df.loc[hui_df['COUNTY'] == '', 'COUNTY'] = np.nan
hui_df.loc[hui_df['SUBCOUNTY'] == '', 'SUBCOUNTY'] = np.nan
hui_df.loc[hui_df['CENTER'] == '', 'CENTER'] = np.nan

In [ ]:
#=========
# Base 
#=========

counties_all = hui_df[hui_df['COUNTY'] != 'None']["COUNTY"].to_list()
subregions_all = hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list()
cities_all = hui_df[hui_df['CITY']!= 'None']["CITY"].to_list()
centers_all = hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list()

counties_all = ["None"] + sorted(list(set(counties_all)))
subregions_all = ["None"] + sorted(list(set(subregions_all)))
cities_all = ["None"] + sorted(list(set(cities_all)))
centers_all = ["None"] + sorted(list(set(centers_all)))


counties_text = str(counties_all).replace("'", '"')
subregions_text = str(subregions_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "BASE.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

f.write("\t{\n")
f.write(f'''\t\t"COUNTY":{counties_text},\n''')
f.write(f'''\t\t"SUBREGION":{subregions_text},\n''')
f.write(f'''\t\t"CITY":{cities_text},\n''')
f.write(f'''\t\t"CENTER":{centers_text}\n''')
f.write("\t}\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#=========
# County 
#=========

counties_all = hui_df[hui_df['COUNTY'] != 'None']["COUNTY"].to_list()
subregions_all = hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list()
cities_all = hui_df[hui_df['CITY']!= 'None']["CITY"].to_list()
centers_all = hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list()

counties_all = ["None"] + sorted(list(set(counties_all)))
subregions_all = ["None"] + sorted(list(set(subregions_all)))
cities_all = ["None"] + sorted(list(set(cities_all)))
centers_all = ["None"] + sorted(list(set(centers_all)))

counties_text = str(counties_all).replace("'", '"')
subregions_text = str(subregions_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')
 
# create and open the json
json = os.path.join(outputs, "COUNTY.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")


for i in counties_all: 
    
    if i != "None":
        df = hui_df[hui_df['COUNTY'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        subregions = ["None"] + sorted(list(set(df[df['SUBCOUNTY']!= 'None']['SUBCOUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df[df['CITY']!= 'None']['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df[df['CENTER']!= 'None']['CENTER'].to_list())))

        # counties_text = str(counties).replace("'", '"')
        subregions_text = str(subregions).replace("'", '"')
        cities_text = str(cities).replace("'", '"')
        centers_text = str(centers).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"SUBREGION":{subregions_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        if i == counties_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#============
# Subregion 
#============

counties_all = hui_df[hui_df['COUNTY'] != 'None']["COUNTY"].to_list()
subregions_all = hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list()
cities_all = hui_df[hui_df['CITY']!= 'None']["CITY"].to_list()
centers_all = hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list()

counties_all = ["None"] + sorted(list(set(counties_all)))
subregions_all = ["None"] + sorted(list(set(subregions_all)))
cities_all = ["None"] + sorted(list(set(cities_all)))
centers_all = ["None"] + sorted(list(set(centers_all)))

counties_text = str(counties_all).replace("'", '"')
subregions_text = str(subregions_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')
 
# create and open the json
json = os.path.join(outputs, "SUBREGION.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")


for i in subregions_all: 
    
    if i != "None":
        df = hui_df[hui_df['SUBCOUNTY'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        subregions = ["None"] + sorted(list(set(df[df['SUBCOUNTY']!= 'None']['SUBCOUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df[df['CITY']!= 'None']['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df[df['CENTER']!= 'None']['CENTER'].to_list())))

        counties_text = str(counties).replace("'", '"')
        cities_text = str(cities).replace("'", '"')
        centers_text = str(centers).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"SUBREGION":{subregions_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        if i == subregions_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
subregions_all

In [ ]:
#=========
# City 
#=========

counties_all = hui_df[hui_df['COUNTY'] != 'None']["COUNTY"].to_list()
subregions_all = hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list()
cities_all = hui_df[hui_df['CITY']!= 'None']["CITY"].to_list()
centers_all = hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list()

counties_all = ["None"] + sorted(list(set(counties_all)))
subregions_all = ["None"] + sorted(list(set(subregions_all)))
cities_all = ["None"] + sorted(list(set(cities_all)))
centers_all = ["None"] + sorted(list(set(centers_all)))


counties_text = str(counties_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "CITY.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")



for i in cities_all: 
    if i != "None":
        df = hui_df[hui_df['CITY'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        subregions = ["None"] + sorted(list(set(df[df['SUBCOUNTY']!= 'None']['SUBCOUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df[df['CENTER']!= 'None']["CENTER"].to_list())))


        counties_text = str(counties).replace("'", '"')
        subregions_text = str(subregions).replace("'", '"')
        centers_text = str(centers).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"SUBREGION":{subregions_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        if i == cities_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
["None"] + list(set(sorted(hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list())))

In [ ]:
#=========
# Center 
#=========

counties_all = hui_df[hui_df['COUNTY'] != 'None']["COUNTY"].to_list()
subregions_all = hui_df[hui_df['SUBCOUNTY']!= 'None']["SUBCOUNTY"].to_list()
cities_all = hui_df[hui_df['CITY']!= 'None']["CITY"].to_list()
centers_all = hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list()

counties_all = ["None"] + sorted(list(set(counties_all)))
subregions_all = ["None"] + sorted(list(set(subregions_all)))
cities_all = ["None"] + sorted(list(set(cities_all)))
centers_all = ["None"] + sorted(list(set(centers_all)))

counties_text = str(counties_all).replace("'", '"')
counties_text = str(subregions_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "CENTER.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

for i in centers_all: 
    
    if i != "None":
        df = hui_df[hui_df['CENTER'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        subregions = ["None"] + sorted(list(set(df[df['SUBCOUNTY']!= 'None']['SUBCOUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df[df['CITY']!= 'None']['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df['CENTER'].to_list())))

        counties_text = str(counties).replace("'", '"')
        subregions_text = str(subregions).replace("'", '"')
        cities_text = str(cities).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"SUBREGION":{subregions_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        
        if i == centers_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#=========
# TYPE
#==========

types = sorted(list(set(hui_df['TYPE'].to_list())),reverse=True)
subtypes = sorted(list(set(hui_df['SUBTYPE'].to_list())))

types_text = str(types).replace("'", '"')
subtypes_text = str(subtypes).replace("'", '"')

json = os.path.join(outputs, "TYPE.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

f.write("\t{\n")
f.write(f'''\t\t"TYPE":{types_text},\n''')
f.write(f'''\t\t"SUBTYPE":{subtypes_text}\n''')
f.write("\t}\n")

# close the json
f.write("]\n")
f.close()